In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install torchmetrics
!pip install segmentation-models-pytorch

In [3]:
import os
import sys
sys.path.append('..')

import os
import torch
import numpy as np
import matplotlib.pyplot as plt
import warnings
from torch.utils.data import DataLoader
from pathlib import Path
warnings.filterwarnings("ignore")

try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# 환경에 따라 PROJECT_ROOT 설정
if IN_COLAB:
    PROJECT_ROOT = '/content/drive/Othercomputers/내 Mac/Road_Lane_segmentation'
else:
    PROJECT_ROOT = Path.cwd().parent

# sys.path 추가 (import용)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# 작업 디렉토리 변경 (파일 접근용)
os.chdir(PROJECT_ROOT)

print(f"Environment: {'Colab' if IN_COLAB else 'Local'}")
print(f"PROJECT_ROOT: {PROJECT_ROOT}")
print(f"Current working directory: {os.getcwd()}")

from src.utils import setup_workspace, load_config, get_device, seed
from src.data import DatasetLoader, get_transforms, split_dataset
from src.models import DeepLabV3Plus
from src.models.losses import get_loss
from src.inference import Inferencer, visualize_prediction, visualize_comparison

Environment: Colab
PROJECT_ROOT: /content/drive/Othercomputers/내 Mac/Road_Lane_segmentation
Current working directory: /content/drive/Othercomputers/내 Mac/Road_Lane_segmentation


In [ ]:
!nvidia-smi

Mon Jan 19 14:56:20 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
SEED = 42
DATA_ROOT = "dataset/Multi Camera Semantic Segmentation"
CONFIG_ROOT = "configs/config.yaml"

setup_workspace(PROJECT_ROOT)
seed(SEED)
get_device()

Env: Colab | Root: /content/drive/Othercomputers/내 Mac/Road_Lane_segmentation
Device: Tesla T4


device(type='cuda')

In [5]:
config = load_config(CONFIG_ROOT)
config

{'data': {'root': 'dataset/Multi_Camera_Semantic_Segmentation',
  'num_classes': 62,
  'img_size': [768, 576]},
 'model': {'name': 'deeplabv3_resnet50', 'pretrained': True},
 'training': {'batch_size': 16,
  'accumulation_steps': 2,
  'epochs': 80,
  'lr': 0.001,
  'weight_decay': 0.0001,
  'dropout': 0.1,
  'early_stop': 10,
  'num_workers': 4},
 'checkpoint': {'dir': 'checkpoints', 'exp_name': 'exp1'},
 'loss': {'type': 'ce+dice',
  'weights': {'ce': 0.5, 'dice': 0.5},
  'ignore_index': 255},
 'wandb': {'enabled': True,
  'project': 'Road_Lane_Segmentation',
  'run_name': 'baseline',
  'tags': ['deeplabv3', 'segmentation'],
  'num_samples': 2}}

In [ ]:
from src.training import Trainer

trainer = Trainer("configs/config.yaml")
trainer.train()

In [ ]:
from src.evaluation import Evaluator

checkpoint_path = "checkpoints/deeplabv3_resnet50_exp1/best.pt"
evaluator = Evaluator(config_path=CONFIG_ROOT, checkpoint_path=checkpoint_path)
metrics = evaluator.evaluate()
metrics

In [7]:
import pandas as pd

metrics_df = pd.DataFrame([metrics])
metrics_df

,miou,accuracy,dice
0,0.381295,0.470973,0.483827


In [8]:
import numpy as np

def create_colormap(num_classes):
    colormap = np.random.randint(0, 255, size=(num_classes, 3), dtype=np.uint8)
    colormap[0] = [0, 0, 0]  # background는 검정
    return colormap

def mask_to_color(mask, colormap):
    h, w = mask.shape
    color_mask = np.zeros((h, w, 3), dtype=np.uint8)
    for class_id, color in enumerate(colormap):
        color_mask[mask == class_id] = color
    return color_mask

In [9]:
MEAN = [0.485, 0.456, 0.406]
STD  = [0.229, 0.224, 0.225]

def visualize_segmentation(
    model,
    dataloader,
    device,
    num_classes,
    num_samples=3,
    alpha=0.5
):
    """모델 추론 결과를 시각화"""
    model.eval()
    colormap = create_colormap(num_classes)

    with torch.no_grad():
        for images, masks in dataloader:
            images = images.to(device)
            masks = masks.to(device)

            outputs = model(images)["out"]
            preds = outputs.argmax(dim=1)

            images = images.cpu()
            masks = masks.cpu()
            preds = preds.cpu()

            for i in range(min(num_samples, images.size(0))):
                img_tensor = images[i].clone()

                # 역정규화 (De-normalization)
                for c in range(3):
                    img_tensor[c] = img_tensor[c] * STD[c] + MEAN[c]

                # Tensor → Numpy Image
                img_np = img_tensor.permute(1, 2, 0).numpy()
                img_np = np.clip(img_np, 0, 1)

                gt_mask = masks[i].numpy()
                pred_mask = preds[i].numpy()

                gt_color = mask_to_color(gt_mask, colormap)
                pred_color = mask_to_color(pred_mask, colormap)

                overlay = img_np * (1 - alpha) + (pred_color / 255.0) * alpha

                fig, axes = plt.subplots(1, 4, figsize=(24, 6))

                axes[0].imshow(img_np)
                axes[0].set_title("Image")

                axes[1].imshow(gt_color)
                axes[1].set_title("GT Mask")

                axes[2].imshow(pred_color)
                axes[2].set_title("Pred Mask")

                axes[3].imshow(overlay)
                axes[3].set_title("Overlay (Prediction)")

                for ax in axes:
                    ax.axis("off")

                plt.tight_layout()
                plt.show()

            break  # 첫 batch만 시각화

In [10]:
visualize_segmentation(
    model=evaluator.model,
    dataloader=evaluator.dataloader,
    device=evaluator.device,
    num_classes=evaluator.num_classes,
    num_samples=3
)

Output hidden; open in https://colab.research.google.com to view.